DETECTOR DE PLAGIO
===============
DETECCION MEDIANTE SENTENCES EMBEDDINGS Y SUS CORRESPONDIENTES BIGRAMAS
---------------

### Contexto
Este trabajo practico fue realizado para la materia Procesamiento del Lenguaje Natural, de la carrera Ingenieria en Sistemas de Informacion, UTN FRBA  
  
### Descripcion
A grandes rasgos podemos distinguir dos grandes modulos:  
 - **Preprocesamiento:** En esta seccion el sistema se encargara de tomar todos los archivos (dataset) que se encuentren en un directorio especificado, extrayendo su texto por medio del parser adecuado para su extension, soportando actualmente .doc | .docx | .pdf.
 - **Deteccion:** Por medio del servicio de deteccion se puede cargar un archivo, el cual sera analizado y contrastado con el set de datos preprocesados con el que se cuenta, pudiendo obtener: procentaje de plagio, oraciones que fueron plagiadas y nombre del archivo del dataset
El proyecto expone dos endpoints:
 - **[POST] /preprocess**, el cual nos permite preprocesar todos los archivos del dataset y tener listo el sistema
 - **[POST] /detect**, el cual nos permite pasarle un archivo por body de la manera -> *file: "nombre_archivo.extension"*. Este mismo debera ser pasado como form-data

### Metodologia  
La metodologia utilizada para realizar la deteccion, es la que se propone en el paper *[An Improved SRL based Plagiarism Detection Technique using
Sentence Ranking](https://www.sciencedirect.com/science/article/pii/S1877050915000794)*, en el que los autores especifican tres pasos o etapas bien definidas
 - Pre-processing *(Pre-procesamiento)*
 - Candidate Retrieval *(Eleccion de candidatos)*
 - Sentence Ranking *(Ranking de oraciones)*
 - Semantic Role Labeling *(Etiquetado semantico de las oraciones)*
 - Similarity Detection *(Deteccion de plagio o similitud)*

#### **Pre-processing (Pre-procesamiento)**
Esta accion inicial nos permite deshacernos de palabras redundantes llamadas 'stopwords' que no aportan contenido al documento o a la idea general.  
  
Para realizar esto utilizaremos la libreria NLTK, con los agregados de las stopwords y punkt, ambas extenciones que nos serviran a la hora de detectar las stopwords y quitarlas

In [1]:
import nltk
from nltk.corpus import stopwords


nltk.download('punkt')
nltk.download('stopwords')

spanish_stopwords = stopwords.words('spanish')

def preprocess_document(text):
    return ' '.join([word.lower() for word in word_tokenize(text) if word.lower() not in spanish_stopwords])

[nltk_data] Downloading package punkt to /home/facundo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/facundo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### **Candidate Retrieval (Eleccion de candidatos)**
En esta seccion lo que se propone es obtener los textos, que podrian haber sido plagiados, de un corpus previamente analizado y preprocesado.  
Este analisis se realizara mediante el metodo de Jaccard el cual nos dara un coeficiente de similitud en base a la siguiente ecuacion:
 - coeficiente = cantidad de intersecciones de bigramas / cantidad de la union de bigramas

In [2]:
def jaccard_similarity_coefficient(suspect_n_grams, original_n_grams):
    return len(suspect_n_grams.intersection(original_n_grams)) / len(suspect_n_grams.union(original_n_grams))


Si bien esta funcion nos permite calcular el coeficiente, aun necesitamos poder calcular los bigramas de un texto.  
Esto lo haremos mediante la combinacion de funcionalidades que nos ofrece la libreria de NLTK.  
Por un lado utilizaremos *word_tokenize* la cual nos dara un array de palabras y por otro lado usaremos *bigrams* el cual nos generara un array de tuplas (bigramas) dado el array anterior

In [3]:
from nltk import bigrams, word_tokenize

def get_text_bygrams(text):
    return set(bigrams(word_tokenize(text)))

Por ultimo, y ya teniendo esto, estamos en condiciones de poder decir si dos textos son plagio.  
Como ultimo dato, necesitaremos un un coeficiente, el cual nos servira como punto de corte para decidir. A este coeficiente le llamaremos *jaccard_threshold*

In [4]:
jaccard_threshold = 0.2

def may_be_plagiarism_of(suspect, original):
    preprocessed_original = preprocess_document(original)
    preprocessed_suspect = preprocess_document(suspect)
    bigrams_original = get_text_bygrams(preprocessed_original)
    bigrams_suspect = get_text_bygrams(preprocessed_suspect)
    coefficient = jaccard_similarity_coefficient(bigrams_suspect, bigrams_original)
    return coefficient > jaccard_threshold

#### **Sentence Ranking (Ranking de oraciones)**
Este paso podemos dividirlo conceptualmente en dos bloques:
 - **Vectorizacion oraciones:** Este paso consiste en calcular los embeddings de una oracion. Esto graficamente seria como asignarle un vector a cada oracion analizada. Nos ayudaremos de un modelo pre entrenado que ofrece google y es parse de Tensorflow. Este se llama *universal-sentence-encoder-multilingual* en su verison 3. Una de las cosas interesantes que nos ofrece es la posibilidad de elegir la version multilenguaje, la cual se adapta al castellano entre otros. 
 

Primero debemos asegurarnos de tener instalado tensorflow en una version mayor a la 2.0.0 y Tensorflow hub.  
Esta informacion y mas, podemos encontrarla en el [apartado oficial de Tensorflow](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 )

In [5]:
!pip3 install tensorflow_text>=2.0.0rc0
!pip install tensorflow_hub
import tensorflow_text
import tensorflow_hub as hub
import numpy as np

Luego podremos obtener el modelo del encoder para calcular los embeddings

In [6]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
# Ejemplo:
# embed('un texto')

 - **Comparacion mediante el metodo del coseno:** Utilizando la ecuacion que se muestra debajo, podemos calcular la similitud entre dos vectores. Expandiendo esto, podemos hacerlo con todo el array de embeddings dado por el modelo pre entrenado de Tensorflow

SKLearn nos provee de esta funcion ya previamente desarrollada, por lo tanto solo basta con importarla y utilizarla

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

Por ultimo nos queda implementar una funcion que calcule los embeddings de ambos textos a comparar y que, con una adaptacion previa, se los pase al servicio de sklearn para que nos devuelva un coeficiente.  
Nuevamente debemos proponer un threshold el cual nos servira para determinar con que nivel de exigencia decimos que una oracion es similar a otra. A este coeficiente lo llamaremos *cosine_similarity_threshold*

In [8]:
cosine_similarity_threshold = 0.7

def sentence_is_semantically_similar(suspect_sentence, original_sentence):
    suspect_embedding = embed(suspect_sentence).numpy().reshape(1, 512)
    original_embedding = embed(original_sentence).numpy().reshape(1, 512)
    return cosine_similarity(suspect_embedding, original_embedding) >= cosine_similarity_threshold

#### **Semantic Role Labeling (Etiquetado semantico de las oraciones)**
Consiste en asignar un proposito a las partes que componen a una oracion. Pudiendo determinar: "qué", "quién", "donde", "cómo", "cuando".
Esta operacion se realizara sobre las obtenidas anteriormente, presentando similitudes relevantes

#### **Similarity Detection (Deteccion de plagio o similitud)**
Por ultimo realizaremos la comparacion entre textos, esto implica separar por oraciones haciendo uso de la funcionalidad *sent_tokenize* que nos ofrece NLTK y comparando contra todo el resto del texto original

In [9]:
from nltk import sent_tokenize

def any_match(collection, function):
    for element in collection:
        if function(element):
            return True
    return False

def sentence_is_plagiarism(suspect_sentence, original_document):
    return any_match(
        sent_tokenize(original_document),
        lambda original_sentence: sentence_is_semantically_similar(suspect_sentence, original_sentence)
    )

def plagiarised_sentences(suspect_document, original_document):
    suspect_sentences = sent_tokenize(suspect_document)
    return list(
        filter(
            lambda suspect_sentence: sentence_is_plagiarism(suspect_sentence, original_document),
            suspect_sentences
        )
    )

El calculo de procentaje de plagio se realiza con una simple relacion entre el total de oraciones y el la cantidad de oraciones que se determinaron como plagio

In [10]:
def plagiarism_percentage(suspect_document, original_document):
    n_plagiarised = len(plagiarised_sentences(suspect_document, original_document))
    n_total = len(sent_tokenize(suspect_document))
    return n_plagiarised / n_total

Finalmente hacemos una prueba con un pequeno parrafo parafraseado

In [11]:
orig = "Hola soy Pablo. Me gustaria una naranja."
plag = "Me gusta la naranja. Él se llama Pablo."
puede_ser_plagio = may_be_plagiarism_of(plag, orig)
porcentaje_de_plagio = plagiarism_percentage(plag, orig)
oraciones_plagio = plagiarised_sentences(plag, orig)
print(f"  - Puede ser plagio: {puede_ser_plagio}.")
print(f"  - Porcentaje de plagio: {porcentaje_de_plagio}.")
print(f"  - Oraciones que realizan plagio: {oraciones_plagio}")

  - Puede ser plagio: True.
  - Porcentaje de plagio: 0.5.
  - Oraciones que realizan plagio: ['Me gusta la naranja.']
